In [1]:
import numpy as np
import pandas as pd
from scipy.stats import f
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.spatial.distance import mahalanobis

In [2]:
# install openpyxl

In [87]:
def get_train_data(data, features, train_samples=None):
    if train_samples:
        train_data = pd.DataFrame()
        
        for cls, samples in train_samples.items():
            train_samps = data.iloc[samples, :]
            train_samps["Class"] = cls
            train_data = pd.concat([train_data, train_samps])
    else:
        assert data.shape[1] == len(features) + 1
        train_data = data.dropna()
        cls_col = train_data.drop(columns=features).columns[0]
        train_data = train_data.rename({cls_col: "Class"}, axis=1)
    train_data = train_data.astype({"Class": 'int32'})
    return train_data

def scatter_matrix(samples):
    # является ли подклассом?
    if isinstance(samples, pd.Series):
        samples = samples.to_frame()
    d = samples - samples.mean()
    res = np.zeros((d.shape[1], d.shape[1]))
    # приводит к виду int: 32, 24, ...
    for _, row in d.iterrows():
        col = row.to_frame()
        res += (col @ col.T).astype(np.float64)
    return res

def classes_scatter_matrix(samples, labels):
    A = np.zeros((samples.shape[1], samples.shape[1]))
    for cls in labels.unique():
        A += scatter_matrix(samples[labels == cls])
    return A

def find_mahl_sqr_dist(centers, samples, covr):
    res = pd.DataFrame(index=samples.index, columns=centers.index)
    for i in centers.index:
        for j in samples.index:
            res[i][j] = mahalanobis(centers.loc[i], samples.loc[j], np.linalg.inv(covr)) ** 2
    return res

def get_def_coef(lda, features):
    return pd.DataFrame(
        np.vstack([lda.intercept_, lda.coef_.T]),
        index=["Const"] + features,
        columns=lda.classes_
    )

def LDA_predict(lda, x):
    return pd.DataFrame(
        lda.predict(x),
        columns=["Class"],
        index=x.index
    )

def LDA_predict_probab(lda, x):
    return pd.DataFrame(
        lda.predict_proba(x),
        columns=lda.classes_,
        index=x.index
    )

def wilks_lambda(samples, labels):
    if isinstance(samples, pd.Series):
        samples = samples.to_frame()
    # определитель матрицы рассеивания
    dT = np.linalg.det(scatter_matrix(samples))
    # определитель классовой матрицы рассеивания
    dE = np.linalg.det(classes_scatter_matrix(samples, labels))
    return dE / dT

def f_p_value(lmbd, n_obj, n_sign, n_cls):
    num = (1-lmbd)*(n_obj - n_cls - n_sign)
    den = lmbd * (n_cls - 1)
    f_value = num / den
    p = f.sf(f_value, n_cls-1, n_obj-n_cls-n_sign)
    return f_value, p

def forward(samples, labels, f_in = 1e-4):
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to enter", "P value"]
    n_cls = labels.unique().size
    n_obj = samples.shape[0]
    # хранение пременных вне и в модели(е)
    out = {0: pd.DataFrame(columns=st_columns, index=samples.columns, dtype=float)}
    into = {0: pd.DataFrame(columns=st_columns, dtype=float)}
    step = 0

    while True:
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        # расчёт характеристик элементов вне модели
        for el in out[step].index:
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [el]], labels)
            partial_lmbd = lmbda / model_lmbd
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        # расчёт характеристик элементов в моделе

        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size-1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        if out[step].index.size == 0 or out[step]["F to enter"].max() < f_in:
            break
        # добавление нового элемента
        el_to_enter = out[step]["F to enter"].idxmax()
        into[step+1] = into[step].append(out[step].loc[el_to_enter])
        out[step+1] = out[step].drop(index=el_to_enter)
        step += 1
    return into, out

def backward(samples, labels, f_r = 1e+4):
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to remove", "P value"]
    n_cls = labels.unique().size
    n_obj = samples.shape[0]
    # хранение пременных вне и в модели(е)
    into = {0: pd.DataFrame(columns=st_columns, index=samples.columns, dtype=float)}
    out = {0: pd.DataFrame(columns=st_columns, dtype=float)}
    step = 0

    while True:
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        # расчёт характеристик элементов вне модели
        for el in out[step].index:
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [el]], labels)
            partial_lmbd = lmbda / model_lmbd
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        # расчёт характеристик элементов в моделе
        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size-1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        if into[step].index.size == 0 or into[step]["F to remove"].min() > f_r:
            break
        # удаление элемента
        el_to_remove = into[step]["F to remove"].idxmin()
        out[step+1] = out[step].append(into[step].loc[el_to_remove])
        into[step+1] = into[step].drop(index=el_to_remove)
        step += 1
    return into, out

In [76]:
FEATURES = ["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "X11"]

TRAIN_SAMPLES = {
    1: [17, 51, 55],
    2: [4, 18, 25],
    3: [5, 6, 8, 23, 24, 28, 47, 63, 65, 84],
    4: [39, 42, 64, 68, 72, 75],
    5: [33, 66, 78],
    6: [14, 48],
    7: [34, 82]
}

In [41]:
data = pd.read_excel('Data.xlsx', sheet_name='Итоговые данные', index_col=0).drop('Субъект Российской Федерации').loc[:, FEATURES]
#data = data.loc[range(0, 85)]
print("Данные")
data_to_excel = data
data.head()

Данные


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
Субъект Российской Федерации,,,,,,,,,,,
Алтайский край,1166,5.8,18.334,17.6,60.7,0.6,0.42,5.76,1.38,4.64,-18
Амурская область,1109,7.1,26.407,15.7,64.1,0.82,0.51,11.54,2.5,9.16,0.2
Архангельская область,1132,6.1,29.187,13.6,60,0.73,0.78,7.04,0.76,4.79,-26
Астраханская область,1122,5.8,21.558,15.5,62.2,0.77,0.84,3.9,1.33,3.9,-78
Белгородская область,1163,6.7,26.072,7.8,67.3,0.31,0.47,3.77,0.69,2.61,60


In [42]:
index = data.index
columns = data.columns
scaled = (data - data.mean(axis=0))/data.std()
scaled_data = pd.DataFrame(scaled, columns=columns, index=index) 
scaled_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
Субъект Российской Федерации,,,,,,,,,,,
Алтайский край,0.480197,-0.517762,-0.796615,0.654258,-0.497539,-0.337127,-1.246125,-0.103236,0.188463,0.058932,-0.468984
Амурская область,-0.613109,0.87767,0.254169,0.28455,0.162648,0.426103,-0.781334,2.234082,2.272277,2.472753,-0.130889
Архангельская область,-0.17195,-0.195739,0.616015,-0.124076,-0.63346,0.113872,0.613038,0.414371,-0.965077,0.139037,-0.617597
Астраханская область,-0.363758,-0.517762,-0.376978,0.245633,-0.20628,0.252642,0.922898,-0.855383,0.095435,-0.336251,-1.583583
Белгородская область,0.422655,0.448306,0.210566,-1.25266,0.784,-1.343204,-0.987908,-0.907952,-1.095316,-1.025152,0.979995


In [43]:
data = scaled_data
data_to_excel = scaled_data

In [44]:
train_data = get_train_data(data, FEATURES, TRAIN_SAMPLES)
print("Обучающая выборка")
data_to_excel["Train sample"] = train_data.Class
train_data

Обучающая выборка


<ipython-input-39-bef845d683dd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_samps["Class"] = cls


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Class
Субъект Российской Федерации,,,,,,,,,,,,
Кабардино-Балкарская \nРеспублика,-0.248673,-1.161808,-0.873279,1.93851,-0.245114,-0.059589,0.561394,-1.308289,0.225674,-0.896984,-0.264641,1
Республика Калмыкия,-1.11181,-1.37649,-1.712683,1.802301,-0.827632,-1.481973,0.406464,-1.348727,-0.202253,-1.206722,-1.342087,1
Республика Марий Эл,-0.018504,-1.161808,-1.102882,1.140717,-0.245114,-0.82282,0.04496,-0.523791,-0.965077,-0.437717,0.014009,1
Белгородская область,0.422655,0.448306,0.210566,-1.25266,0.784,-1.343204,-0.987908,-0.907952,-1.095316,-1.025152,0.979995,2
Калининградская область,-0.287035,1.199693,-0.093489,-0.124076,0.764583,-0.441204,-0.574761,0.309232,-0.685995,0.507518,2.261784,2
Краснодарский край,0.30757,1.307034,1.106719,-0.688368,0.104396,-0.961589,0.04496,0.06256,-0.258069,0.53956,1.054301,2
Брянская область,0.863813,0.233624,0.002049,-0.085159,-0.148028,-0.68405,-0.006683,-0.584448,-0.927866,-0.218764,-0.140177,3
Владимирская область,1.170706,-0.30308,-0.461973,-0.318659,0.570411,-0.649358,-0.006683,-0.273076,-0.87205,-0.346932,0.236929,3
Вологодская область,0.537739,-0.410421,-0.447004,-0.260284,-0.361618,-0.718743,-0.41983,0.107042,-0.909261,0.02155,-0.468984,3


In [88]:
cov = pd.DataFrame(
    classes_scatter_matrix(train_data[FEATURES], train_data.Class) / (train_data.shape[0] - train_data.Class.unique().size),
    index=FEATURES,
    columns=FEATURES
)

print("Ковариационная матрица")
cov

Ковариационная матрица


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
X1,0.238453,-0.080786,0.035025,-0.062652,-0.032753,0.043872,0.150030,0.046707,-0.029733,-0.012840,-0.030049
X2,-0.080786,0.270455,0.082067,-0.028491,0.171971,0.008610,-0.121141,-0.048949,-0.037737,0.076978,0.172236
X3,0.035025,0.082067,0.195899,-0.052575,-0.026962,0.050384,0.042359,0.070211,0.011988,0.144377,0.046098
X4,-0.062652,-0.028491,-0.052575,0.116481,-0.044134,0.005713,0.012001,-0.002731,0.034151,0.005509,-0.042550
X5,-0.032753,0.171971,-0.026962,-0.044134,0.397871,-0.027631,-0.175043,-0.062418,-0.093948,-0.019674,0.148824
X6,0.043872,0.008610,0.050384,0.005713,-0.027631,0.216727,0.067420,0.034703,0.068979,0.029138,-0.000767
X7,0.150030,-0.121141,0.042359,0.012001,-0.175043,0.067420,0.289483,0.063432,0.057235,0.003644,-0.112045
X8,0.046707,-0.048949,0.070211,-0.002731,-0.062418,0.034703,0.063432,0.270976,0.020161,0.207537,-0.043018
X9,-0.029733,-0.037737,0.011988,0.034151,-0.093948,0.068979,0.057235,0.020161,0.137730,0.013100,-0.075364
X10,-0.012840,0.076978,0.144377,0.005509,-0.019674,0.029138,0.003644,0.207537,0.013100,0.285673,0.068101


In [90]:
lda = LinearDiscriminantAnalysis().fit(train_data[FEATURES], train_data.Class)
means = pd.DataFrame(lda.means_, index=lda.classes_, columns=FEATURES)
print("Средние значения")
means

Средние значения


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
1,-0.459662,-1.233368,-1.229615,1.627176,-0.439287,-0.788127,0.337606,-1.060269,-0.313885,-0.847141,-0.530906
2,0.147730,0.985011,0.407932,-0.688368,0.550993,-0.915332,-0.505903,-0.178720,-0.679793,0.007309,1.432027
3,0.794762,-0.292346,-0.336264,-0.297255,-0.132494,-0.344066,-0.259736,-0.226976,-0.561338,-0.325571,-0.124016
4,0.342735,-0.070508,-0.271526,-0.026784,-0.128611,0.784590,0.303177,0.168373,0.504756,-0.120859,-0.078874
5,-1.405915,1.092352,1.729146,-0.973757,1.152928,0.599564,-0.437045,0.530293,0.914076,0.147937,-0.667135
6,-0.967954,0.233624,-0.481432,1.276926,-0.507247,-0.475897,-0.755513,1.504175,0.346609,2.227099,-0.515426
7,-2.569551,-0.088398,0.957295,-1.028889,1.638360,0.009795,2.498022,-0.707784,-1.169738,-0.208084,1.063590


In [92]:
cen_dis = find_mahl_sqr_dist(means, means, cov)
print("Расстояние Махланобиса (обучающая выборка)")
cen_dis

Расстояние Махланобиса (обучающая выборка)


,1,2,3,4,5,6,7
1,0.0,75.442165,47.943674,57.921947,178.296922,69.35387,408.183495
2,75.442165,0.0,23.935338,29.410213,88.149767,89.650041,414.330062
3,47.943674,23.935338,0.0,16.074777,125.860266,55.664155,462.5482
4,57.921947,29.410213,16.074777,0.0,90.783984,56.924973,426.378506
5,178.296922,88.149767,125.860266,90.783984,0.0,189.215869,336.339811
6,69.35387,89.650041,55.664155,56.924973,189.215869,0.0,536.395831
7,408.183495,414.330062,462.5482,426.378506,336.339811,536.395831,0.0


In [94]:
df_coef = get_def_coef(lda, FEATURES)
print("Функции Фишера")
df_coef

Функции Фишера


,1,2,3,4,5,6,7
Const,-21.583691,-10.314893,-5.537145,-4.371980,-41.693641,-28.146974,-195.681478
X1,1.830658,4.090187,8.966840,4.751564,-12.318491,6.674442,-56.166865
X2,-6.856695,5.825405,1.765559,3.347666,1.786229,4.053237,-24.056439
X3,1.346871,4.906811,-3.557905,-4.120255,21.054379,-13.717516,2.905715
X4,16.359114,2.146483,0.625253,0.322121,-3.170633,5.701018,-32.796092
X5,1.380023,-2.809494,-3.543091,-1.393413,5.997087,-3.034163,18.078429
X6,-2.068015,-6.352004,-0.383959,2.351345,-0.814825,-1.462769,10.180795
X7,0.376896,-2.647436,-6.397229,-2.259818,0.940500,-5.688509,46.449170
X8,-7.508434,8.263278,-2.852616,2.192554,17.334103,-6.621633,-12.826369
X9,-3.772555,2.941764,-0.777283,6.069369,7.730310,3.140792,-27.811762


In [96]:
print("Pi: ", *lda.priors_)

Pi:  0.10344827586206896 0.10344827586206896 0.3448275862068966 0.20689655172413793 0.10344827586206896 0.06896551724137931 0.06896551724137931


In [97]:
lda_predict = LDA_predict(lda, data[FEATURES])
print("Распределение по классам")
print(lda_predict)
data_to_excel["Result Lda"] = lda_predict

Распределение по классам
                                   Class
Субъект Российской Федерации            
Алтайский край                         3
Амурская область                       6
Архангельская область                  3
Астраханская область                   3
Белгородская область                   2
...                                  ...
Чеченская Республика                   1
Чувашская Республика                   1
Чукотский автономный округ             7
Ямало-Ненецкий автономный \nокруг      5
Ярославская область                    3

[85 rows x 1 columns]


In [99]:
samp_dist = find_mahl_sqr_dist(means, data[FEATURES], cov)
print("Расстояние Махланобиса")
samp_dist

Расстояние Махланобиса


,1,2,3,4,5,6,7
Субъект Российской Федерации,,,,,,,
Алтайский край,66.030418,70.983317,27.997298,45.510312,194.72405,45.659499,666.938981
Амурская область,182.294314,119.038044,115.669811,76.956768,159.092148,69.941129,665.296235
Архангельская область,65.542573,49.956161,44.382956,51.266208,86.726632,101.836702,292.551711
Астраханская область,58.911239,97.650509,55.585409,54.895736,138.420992,80.242468,295.448577
Белгородская область,90.231148,10.686306,26.999827,47.569128,93.486079,126.872352,425.97497
...,...,...,...,...,...,...,...
Чеченская Республика,65.646649,123.120628,120.029737,157.587031,205.456866,183.959696,401.973865
Чувашская Республика,45.672405,113.291539,75.217397,77.316235,191.083667,118.208479,245.238258
Чукотский автономный округ,414.406867,405.464627,465.032023,431.871054,321.813778,536.413296,9.061154


In [100]:
lda_post_prob = LDA_predict_probab(lda, data[FEATURES])
print("Probabilities")
lda_post_prob

Probabilities


,1,2,3,4,5,6,7
Субъект Российской Федерации,,,,,,,
Алтайский край,1.653028e-09,1.389223e-10,9.998763e-01,9.444844e-05,1.874188e-37,2.921985e-05,3.602074e-140
Амурская область,5.514751e-25,3.002275e-11,5.391773e-10,8.247327e-02,6.022991e-20,9.175267e-01,4.818369e-130
Архангельская область,7.350133e-06,1.781729e-02,9.636657e-01,1.850967e-02,1.846015e-10,6.442196e-14,2.487688e-55
Астраханская область,2.987273e-02,1.156510e-10,5.252288e-01,4.448980e-01,1.621629e-19,4.646932e-07,8.624888e-54
Белгородская область,5.328955e-18,9.990449e-01,9.550536e-04,1.957095e-08,1.046750e-18,3.926580e-26,4.412718e-91
...,...,...,...,...,...,...,...
Чеченская Республика,1.000000e+00,3.308903e-13,5.172984e-12,2.169850e-20,4.371164e-31,1.356928e-26,6.185598e-74
Чувашская Республика,9.999985e-01,2.073438e-15,1.280161e-06,2.689422e-07,2.656715e-32,1.182768e-16,3.081298e-44
Чукотский автономный округ,1.433468e-88,1.253634e-86,4.854604e-99,4.625067e-92,1.831142e-68,3.068580e-115,1.000000e+00


In [102]:
into, out = forward(train_data[FEATURES], train_data.Class)
print("Forward stepwise")
for i, tab in into.items():
    print("Step: ", i)
    print(tab, end="\n\n")

forw_stepwise = into[len(into) - 4].index.tolist()
print(forw_stepwise)
forw_stepwise_lda = LinearDiscriminantAnalysis().fit(train_data[forw_stepwise], train_data.Class)
forw_stepwise_coef = get_def_coef(forw_stepwise_lda, forw_stepwise)
print("Функции Фишера")
print(forw_stepwise_coef)
print("Pi: ", forw_stepwise_lda.priors_)
forw_stepwise_pred = LDA_predict(forw_stepwise_lda, data[forw_stepwise])
print("Распределение")
print(forw_stepwise_pred.head())
data_to_excel["Result forward"] = forw_stepwise_pred

Forward stepwise
Step:  0
Empty DataFrame
Columns: [Wilk's lmbd, Partial lmbd, F to enter, P value]
Index: []

Step:  1
    Wilk's lmbd  Partial lmbd  F to enter       P value
X4          1.0      0.123327   26.064702  6.159773e-09

Step:  2
    Wilk's lmbd  Partial lmbd  F to enter       P value
X4     0.155184      0.105918   29.544497  3.391923e-09
X1     0.123327      0.133278   22.760865  3.580205e-08

Step:  3
    Wilk's lmbd  Partial lmbd  F to enter       P value
X4     0.031964      0.092367   32.754544  2.503616e-09
X1     0.034492      0.085596   35.609157  1.185103e-09
X7     0.016437      0.179621   15.224290  1.615824e-06

Step:  4
    Wilk's lmbd  Partial lmbd  F to enter       P value
X4     0.005535      0.093451   30.719056  8.408574e-09
X1     0.006592      0.078462   37.192728  1.645016e-09
X7     0.003363      0.153816   17.420732  8.458774e-07
X9     0.002952      0.175184   14.909565  2.781659e-06

Step:  5
     Wilk's lmbd  Partial lmbd  F to enter       P value

In [104]:
into, out = backward(train_data[FEATURES], train_data.Class)
print("Backward stepwise")
for i, tab in into.items():
    print("Step: ", i)
    print(tab, end="\n\n")

back_stepwise = into[len(into) - 7].index.tolist()
print(back_stepwise)
back_stepwise_lda = LinearDiscriminantAnalysis().fit(train_data[back_stepwise], train_data.Class)
back_stepwise_coef = get_def_coef(back_stepwise_lda, back_stepwise)
print("Функции Фишера")
print(back_stepwise_coef)
print("Pi: ", back_stepwise_lda.priors_)
back_stepwise_pred = LDA_predict(back_stepwise_lda, data[back_stepwise])
print("Распределение")
print(back_stepwise_pred.head())
data_to_excel["Result backward"] = back_stepwise_pred
data_to_excel.to_excel(r'Data_py.xlsx')

Backward stepwise
Step:  0
     Wilk's lmbd  Partial lmbd  F to remove   P value
X1      0.000054      0.178014     9.235069  0.000640
X2      0.000014      0.684744     0.920798  0.513194
X3      0.000026      0.370156     3.403122  0.033722
X4      0.000026      0.367268     3.445619  0.032404
X5      0.000013      0.726922     0.751327  0.620352
X6      0.000016      0.605402     1.303588  0.326787
X7      0.000033      0.287670     4.952400  0.009028
X8      0.000016      0.610305     1.277048  0.337299
X9      0.000017      0.561977     1.558867  0.241204
X10     0.000024      0.395480     3.057150  0.047065
X11     0.000017      0.556041     1.596855  0.230619

Step:  1
     Wilk's lmbd  Partial lmbd  F to remove   P value
X1      0.000106      0.124263    15.269537  0.000033
X2      0.000017      0.753467     0.708931  0.648629
X3      0.000035      0.380798     3.523134  0.027039
X4      0.000036      0.363141     3.799796  0.020765
X6      0.000023      0.576074     1.594424  